In [1]:
%matplotlib inline
from matplotlib.ticker import FuncFormatter
import matplotlib.patches as mpatches
import matplotlib.pyplot as plt
import pandas as pd
import mpld3
import json
import furl

In [2]:
input_csv = '../../data/mock/wog_overview_level1.csv'
df = pd.read_csv(input_csv)

In [3]:
print(df)

  Service Classification  Sum of Total Financial Year
0     Corporate Services     500000000        2014-15
1     Corporate Services     600000000        2015-16
2            HR Services     300000000        2014-15
3            HR Services     340000000        2015-16
4     Financial Services     230000000        2014-15
5     Financial Services     300000000        2015-16
6         Major Projects      50000000        2014-15
7         Major Projects      60000000        2015-16
8                    ERP      70000000        2014-15
9                    ERP      30000000        2015-16


In [4]:
class XLabelLink(mpld3.plugins.PluginBase):
    """XLabelLink plugin"""
    
    JAVASCRIPT = """
    mpld3.register_plugin("xlabellink", XLabelLink);
    XLabelLink.prototype = Object.create(mpld3.Plugin.prototype);
    XLabelLink.prototype.constructor = XLabelLink;
    XLabelLink.prototype.defaultProps = {urls: null}
    function XLabelLink(fig, props){
        mpld3.Plugin.call(this, fig, props);
    };
    
    XLabelLink.prototype.draw = function(){
        var urls = $.parseJSON(this.props['urls']);
        var labels = d3.selectAll("g.mpld3-xaxis g.tick text");
        labels.each(function() {
            var label = $(this);
            label.attr('style', label.attr('style') + 'cursor: hand;');
            label.on('click', function(){
                var url = urls[label.text()];
                console.log(label.text(), url);
                window.open(url);
            });
        })
    }
    """
    def __init__(self, urls_map):
        self.dict_ = {
            "type": "xlabellink",
            "urls": json.dumps(urls_map)
        }

In [5]:
class YTickFormat(mpld3.plugins.PluginBase):
    """YTickFormat plugin"""
    
    JAVASCRIPT = """
    mpld3.register_plugin("ytickformat", YTickFormat);
    YTickFormat.prototype = Object.create(mpld3.Plugin.prototype);
    YTickFormat.prototype.constructor = YTickFormat;
    YTickFormat.prototype.defaultProps = {formatting: null}
    function YTickFormat(fig, props){
        mpld3.Plugin.call(this, fig, props);
    };
    
    YTickFormat.prototype.draw = function(){
        // FIXME: this is a very brittle way to select the y-axis element
        var ax = this.fig.axes[0].elements[1];
        var formatting = this.props['formatting'];
        var format_func = d3.format("d");
        if (formatting == 'dollar string') {
            format_func = function(d) { return "$" + d3.format(",.2s")(d); };
        }
        ax.axis.tickFormat(format_func);
        
        // HACK: use reset to redraw figure
        this.fig.reset();
    }
    """
    
    def __init__(self, formatting):
        self.dict_ = {
            "type": "ytickformat",
            "formatting": formatting
        }

In [6]:
# Initialize figure and axes
fig, ax = plt.subplots(nrows=1, ncols=1)

pos = range(len(df[df['Financial Year'] == '2015-16']))
width = 0.25

group1 = df[df['Financial Year'] == '2014-15']['Sum of Total']
group2 = df[df['Financial Year'] == '2015-16']['Sum of Total']

bars1 = ax.bar([x - 0.13 for x in pos], group1, color='#579cd0', width=width, linewidth=4, edgecolor='#579cd0', zorder=2)
bars2 = ax.bar([x + 0.13 for x in pos], group2, color='#f17c40', width=width, linewidth=4, edgecolor='#579cd0', zorder=2)

# Set chart properties and formatting
fig.set_size_inches(15,6)
ax.tick_params(axis='y', which='major', labelsize=14, labelcolor='#4373bf', length=0)
ax.tick_params(axis='x', which='major', labelsize=14, labelcolor='#1c4f76', length=0)
ax.set_xticks([p for p in pos])
xlabels = df['Service classification'].unique()
ax.set_xticklabels(xlabels, fontweight='semibold', position=(0,-0.04))

for i, bar in enumerate(bars1.get_children()):
    value = list(group1)[i]
    tooltip = mpld3.plugins.LineLabelTooltip(bar, label='${:,.0f}'.format(value))
    mpld3.plugins.connect(plt.gcf(), tooltip)
    
for i, bar in enumerate(bars2.get_children()):
    value = list(group2)[i]
    tooltip = mpld3.plugins.LineLabelTooltip(bar, label='${:,.0f}'.format(value))
    mpld3.plugins.connect(plt.gcf(), tooltip)

ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['left'].set_visible(False)
ax.spines['bottom'].set_visible(False)
ax.yaxis.grid(color='lightgray', linewidth=2)

# Draw the legend
patch1 = mpatches.Patch(color='#579cd0', label='2014-15', linewidth=4)
patch2 = mpatches.Patch(color='#f17c40', label='2015-16', linewidth=4)
ax.legend(handles=[patch1, patch2], frameon=False, loc='center right')

# Put URLs in x-axis labels
base_url = 'https://www-integration.benchmarking.fin.gosource.com.au/chart/whole-government/level2/'

filters = {
  "period": [
    "15/16FY",
    "14/15FY"
  ],
  "agency": [
    "Org1",
    "Org2"
  ],
  "service_level1": [
    "Financial Services"
  ],
  "service_level2": ["All"]
}

def build_url(service_level1):
    f = furl.furl('')
    filters['service_level1'] = service_level1
    f.args = filters
    return base_url + f.url + '&search='
    
urls_map = {k: build_url(k) for k in xlabels}
mpld3.plugins.connect(fig, XLabelLink(urls_map))

mpld3.plugins.connect(fig, YTickFormat('dollar string'))

# Display the figure
mpld3.display(fig)